In [ ]:
import os

import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
%%time
films = pd.read_csv('assets/title.basics.tsv.gz', sep='\t', na_values=['\\N'])
print(f'{films.shape=}')

ratings = pd.read_csv('assets/title.ratings.tsv.gz', sep='\t', na_values=['\\N'])
print(f'{ratings.shape=}')

# будем рассматривать только фильмы с оценками
films = pd.merge(films, ratings, on='tconst', how='inner')

crew = pd.read_csv('assets/title.crew.tsv.gz', sep='\t', na_values=['\\N'])
print(f'{crew.shape=}')

names = pd.read_csv('assets/name.basics.tsv.gz', sep='\t', na_values=['\\N'])
print(f'{names.shape=}')


In [ ]:
films.head()

In [ ]:
films.startYear.hist()

### ограничивания датасет выборкой из 10^5 фильмов, чтобы выполнять команды было не долго
Когда станет понятно, что нужно посчитать, надо будет на полном датасете считать

In [ ]:
df = films.sample(100000)
df.shape

In [ ]:
df = pd.merge(df, crew, how='left', on='tconst')
df.head()

In [ ]:
# у фильма может быть несколько режиссеров
# строка с запятыми будет преобразована в список строк
df.directors = df.directors.apply(lambda x: x.split(',') if not pd.isna(x) else np.nan)
df.writers = df.writers.apply(lambda x: x.split(',') if not pd.isna(x) else np.nan)

df['number_of_directors'] = df.directors.apply(lambda x: len(x) if not np.all(pd.isna(x)) else np.nan)

# у фильма может быть несколько жанров (до трех)
# строка с запятыми будет преобразована в список строк
# main_genre это первый жанр
df['genres_lst'] = df.genres.apply(lambda x: 
                                   x.split(',') if not pd.isna(x) else np.nan)

df['main_genre'] = df.genres_lst.apply(lambda x: 
                                       x[0] if not np.all(pd.isna(x)) else np.nan)

df.head()

In [ ]:
sns.boxplot(df, x='main_genre', y='averageRating', hue='isAdult')
plt.xticks(rotation=90);

In [ ]:
sns.boxplot(df, x='main_genre', y='numVotes', hue='isAdult')
plt.xticks(rotation=90);
plt.yscale('log')

In [ ]:
# пусть в каждой строке будет указан только один режиссер
# тогда фильму с двумя режиссерами, будет соответствовать две строки
exploded = df.explode('directors')
print(f'{exploded.shape=}')
exploded.head()

In [ ]:
# добавляется информация о человеке
films_with_directors = pd.merge(exploded, names, how='left', 
                                left_on=['directors'], right_on=['nconst'])
print(f'{films_with_directors.shape=}')
films_with_directors.head()

In [ ]:
# сколько режиссеров сняли по какому количеству фильмов
# индекс это количество фильмов
# directors это сколько режиссеров сняли такое количество фильмов
films_with_directors.groupby('directors').averageRating.count().reset_index().groupby('averageRating').count()

In [ ]:
films_with_directors.info()

нулевых значений в поле `genres` не очень много, можно было бы их и удалить

In [ ]:
# может быть жанр фильма закодировать?
pd.get_dummies(films_with_directors.main_genre)

In [ ]:
films_with_directors.head()

In [ ]:
%%time
# создается dataframe для режиссера
aggregated = films_with_directors.groupby(['directors'] + list(names.columns)).agg(list).reset_index()
print(f'{aggregated.shape=}')
aggregated.head()

In [ ]:
from collections import Counter

In [ ]:
# на большом датасете apply может быть недостаточно производительным
# возможно, стоит заменить на groupby + agg, а потом merge
aggregated['average_score'] = aggregated.averageRating.apply(np.mean)
aggregated['most_common_genre'] = aggregated.main_genre.apply(lambda x: Counter(x).most_common()[0][0])

In [ ]:
if 'writers' in aggregated.columns:
    aggregated = aggregated.drop(columns=['writers', 'genres_lst'])

In [ ]:
aggregated.head()

# Вопросы:
1. Режиссер чаще снимает один или с кем-то?
2. Какой лучший фильм у режиссера?
3. Какая средняя оценка фильмов для самого частого жанра у этого режиссера?
4. Как составить рейтинг режиссера, чтобы учесть популярность фильма, оценку и жанр? Нужно ли учитывать год?
5. Как у режиссера может быть очень много фильмов?

In [ ]:
aggregated['best_movie'] = aggregated.apply(lambda x: x.primaryTitle[np.argmax(x.averageRating)], axis=1)
aggregated['most_viewed'] = aggregated.apply(lambda x: x.primaryTitle[np.argmax(x.numVotes)], axis=1)

In [ ]:
aggregated.head()

In [ ]:
aggregated.query('nconst == "nm0000005"')['primaryTitle'].values